In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import torchvision.transforms as transform
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision.models as models
import argparse
# my own library
import load_data_1
import net

In [2]:
model = "model300.pkl"
lr = 0.1
epoches = 2000
mini_batch_size = 64
step = 500
load_model = False
adjust_lr = True
crop_size = 41

In [3]:
hr_transform = transform.Compose([
    transform.ToTensor(),
])

img_transform = transform.Compose([
    transform.RandomCrop((crop_size, crop_size)),
    transform.RandomHorizontalFlip(),
    transform.RandomVerticalFlip(),
    transform.RandomRotation(0, 360),
    transform.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5,hue=0.5)
])

In [4]:
if load_model == True:
    network = torch.load('./model/'+model)
else:
    network = net.Net()
print(network)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
network.to(device)

Net(
  (residual_layer): Sequential(
    (0): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (1): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (2): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (3): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (4): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (5): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (6): Conv_ReLU_Bl

Net(
  (residual_layer): Sequential(
    (0): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (1): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (2): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (3): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (4): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (5): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (6): Conv_ReLU_Bl

In [5]:
train_set = load_data_1.Load_traindata(transform=img_transform, hr_transform=hr_transform, crop_size=crop_size)
train_load = Data.DataLoader(dataset=train_set, batch_size=mini_batch_size, shuffle=True)

In [6]:
# optimizer = torch.optim.Adam(network.parameters(), lr=lr, weight_decay=0.0001)
optimizer = torch.optim.SGD(network.parameters(), lr=lr, momentum=0.9, weight_decay=0.0001)
# optimizer = torch.optim.RMSprop(network.parameters(), lr=lr)
loss_func = nn.MSELoss()
loss_func1 = nn.MSELoss(reduction="sum")
scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=step, gamma=0.1)

In [7]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [8]:
for epoch in range(epoches):
    print(epoch)
    network.train()
    if epoch > 3*step:
        adjust_lr = False
    if adjust_lr == True:
        scheduler.step()
    for step, (x, y) in enumerate(train_load):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output, r_out = network(x)
        r = y - x
        loss = loss_func(r, r_out)
        loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 0.4) 
        optimizer.step()
    print(loss)
    if (epoch % 20) == 0:
        torch.save(network, './model/model{}.pkl'.format(epoch))
# save model
torch.save(network, './model/model.pkl')

rad_fn=<MeanBackward0>)
tensor(829.6862, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.0062, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(2011.5074, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.0029, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(515.8038, device='cuda:0', grad_fn=<SumBackward0>)
1539
tensor(0.0031, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1004.2598, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.0023, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(756.8230, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.0035, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1141.3284, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.0045, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1445.3673, device='cuda:0', grad_fn=<SumBackward0>)
tensor(0.0043, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(753.1714, device='cuda:0', grad_fn=<SumBackward0>)
1540
tensor(0.0034, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1106.7509, device='cuda:0', grad_fn

TypeError: object of type 'NoneType' has no len()